In [2]:
!pip install pqcrypto

In [11]:
import base64
from pqcrypto.kem import ml_kem_768
from Crypto.Cipher import AES

def hybrid_encrypt(public_key, plaintext):
    # 1. ML-KEM Encapsulation: Generates a 32-byte shared secret and a ciphertext
    kem_ciphertext, shared_secret = ml_kem_768.encrypt(public_key)
    
    # 2. AES-GCM Encryption: Use the shared secret to encrypt the string
    # We use GCM mode because it provides both encryption and integrity (AEAD)
    cipher = AES.new(shared_secret, AES.MODE_GCM)
    ciphertext_bytes, tag = cipher.encrypt_and_digest(plaintext.encode('utf-8'))
    
    # Return everything needed for decryption
    return {
        "kem_ciphertext": kem_ciphertext,
        "aes_nonce": cipher.nonce,
        "aes_tag": tag,
        "aes_ciphertext": ciphertext_bytes
    }

def hybrid_decrypt(secret_key, package):
    # 1. ML-KEM Decapsulation: Recover the shared secret using the private key
    recovered_secret = ml_kem_768.decrypt(secret_key, package["kem_ciphertext"])
    
    # 2. AES-GCM Decryption: Use the recovered secret to decrypt the string
    cipher = AES.new(recovered_secret, AES.MODE_GCM, nonce=package["aes_nonce"])
    decrypted_bytes = cipher.decrypt_and_verify(package["aes_ciphertext"], package["aes_tag"])
    
    return decrypted_bytes.decode('utf-8')

# --- EXECUTION FLOW ---

# 1. Setup Alice's Keys
alice_pub, alice_priv = ml_kem_768.generate_keypair()

# 2. THE BEGINNING: Encryption
secret_message = "Fear thou not; for I am with thee: be not dismayed; for I am thy God: I will strengthen thee; yea, I will help thee; yea, I will uphold thee with the right hand of my righteousness."
print(f"Original Message: {secret_message}\n")

encrypted_package = hybrid_encrypt(alice_pub, secret_message)
print(f"Encrypted Package Size: {len(encrypted_package['aes_ciphertext'])} bytes\n")

print(f"Encrypted Message: {encrypted_package['aes_ciphertext']}\n")
print(f"Encrypted Message in hex: {encrypted_package['aes_ciphertext'].hex()}\n")
print(f"Encrypted Message in base64: {base64.b64encode(encrypted_package['aes_ciphertext']).decode('utf-8')}\n")

# 3. THE END: Decryption
decrypted_message = hybrid_decrypt(alice_priv, encrypted_package)

print(f"\nDecrypted Result: {decrypted_message}\n")
if secret_message == decrypted_message:
    print("✅ Success: String encrypted and decrypted perfectly!")

Original Message: Fear thou not; for I am with thee: be not dismayed; for I am thy God: I will strengthen thee; yea, I will help thee; yea, I will uphold thee with the right hand of my righteousness.

Encrypted Package Size: 181 bytes

Encrypted Message: b'(\xbf\xbc\xf6\xe33\xfd)+[G\xcb/\x1b[<\xc4m\x84;U\xc2\xa0\xfb\x19\xb6\xeaA\xb2\xe4G\xcfk\x03}\x1a;\xb5\xcd\xea\xe28\xb8\\\x9e\xa2\x0e#\x9b@\x81M\xee\xa7\xc9\xee\x8b~\xfc\x95\x1d/\xa8\xf7Ih\xf4`dQ\xf577\xd2\xa5\xb6\xd0a\x18\xee\xf3\x9a;gv\x90S3\x03\xcb\x13d\xc3\xaf-,%oX\xfd\x17\xc8\xab\xbbG\x88\xeet\r\x8c\x0f\x9d\x19\x14<\xd6\x8b\xa8\xac\xd8\xfb\xbc\x0cxk\x98\xd1\x8bKb\xd3\x81.\x1adm\xe7Ac#\xd0\xfb\xe7;\x86&\x19c\xd7q\xfc\xc4\x90\xf0\x08V!\xfd/\x83\xcbV\x83N-\x15\xc1\x01\xa52\x89\x14<C\x8c\xcb\xa8\x96\xa9B\xa8'

Encrypted Message in hex: 28bfbcf6e333fd292b5b47cb2f1b5b3cc46d843b55c2a0fb19b6ea41b2e447cf6b037d1a3bb5cdeae238b85c9ea20e239b40814deea7c9ee8b7efc951d2fa8f74968f4606451f53737d2a5b6d06118eef39a3b677690533303cb1364c3af2d2c256f58fd1